In [20]:
a = [1,0,0]
b= [1,1,0]
if(a ==b):
    print ("True")

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Reshape, Conv2D, MaxPooling2D, Flatten
from keras.preprocessing.sequence import pad_sequences
import numpy as np
from sklearn.metrics import confusion_matrix
import random
import timeit
from sklearn.utils import class_weight
start = timeit.default_timer()


#Constant
EMOTION_ANNOTATORS = {'anger': 0, 'happiness' : 1, 'sadness' : 2, 'neutral' : 3, 'frustration' : 4, 'excited': 5,
           'fear' : 6,'surprise' : 7,'disgust' : 8, 'other' : 9}

EMOTION = {'ang': 0, 'hap' : 1, 'sad' : 2, 'neu' : 3, 'fru' : 4, 'exc': 5,
           'fea' : 6,'sur' : 7,'dis' : 8, 'oth' : 9, 'xxx':10}

METHOD = {'audio_feature':0, 'LSTM':1, 'DRCNN' : 2}

#Method for classification
method = METHOD['DRCNN']

#If data is processed and saved into files, just reload, dont need to re-process
isRawDataProcessed = True

#Development mode. Only run with small data.
dev = False








In [ ]:
import os
from os.path import basename
import cv2

##Load the array of input files and output lables. 
path_processed_data = "processed-data"
dirs_data = ['ang', 'hap', 'sad', 'neu', 'fru', 'exc', 'fea', 'sur']

image_paths = []
labels = []
for dir in dirs_data:
    for f in os.listdir(path_processed_data + "/" + dir):
                if not f.startswith(".") and os.path.splitext(f)[1] == ".png":
                    image_paths.append(path_processed_data + "/" + dir + "/" + f)
                    labels.append(EMOTION[dir])


print ("Finish load image_paths and lables!!")
print ("Size data: ", len(image_paths))



#Shuffle data and split into train and test set
c = list(zip(image_paths, labels))
random.shuffle(c)
image_paths, labels = zip(*c)

num_sample_train_data = int(0.95 * len(image_paths))
train_image_paths = image_paths[0:num_sample_train_data]
train_labels = labels[0:num_sample_train_data]


valid_test_image_paths = image_paths[num_sample_train_data:]
valid_test_labels = labels[num_sample_train_data:]

number_sample_valid_data = int(0.6 * len(valid_test_image_paths))
valid_image_paths = valid_test_image_paths[0:number_sample_valid_data]
valid_labels = valid_test_labels[0:number_sample_valid_data]

test_image_paths = valid_test_image_paths[number_sample_valid_data:]
test_labels = valid_test_labels[number_sample_valid_data:]


    


In [ ]:
num_class = 8
batch_size = 512
width_image = 256
channels = 3 #RGB
epoch = 20


def generate_batch_data(data_in, data_out, batch_size):
    while True:
        for i in range(0, len(data_in), batch_size):
            # create Numpy arrays of input data
            # and labels, from each line in the file
            x, y = process_data(data_in[i:i+batch_size], data_out[i:i+batch_size])
            yield (x, y)


##Function to transform these above array into data used for training and testing
def process_data(image_paths, labels):
    input = []
    output = []
    for path,label in zip(image_paths, labels):
        img = cv2.imread(path)
        #img = cv2.resize(img, (256, 256))
        input.append(img)
        
        one_hot_coding_output = np.zeros((1,num_class))[0]
        one_hot_coding_output[label] = 1
        output.append(one_hot_coding_output)
    
    input = np.array(input)
    output = np.array(output)
    return input, output


if method == METHOD['DRCNN']: 
    
       
    model = Sequential()
    
    #C1,P1
    model.add(Conv2D(4, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(width_image, width_image, channels)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #C2,P2
    model.add(Conv2D(8, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    #C3
    model.add(Conv2D(16, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu'))
    
    #C4
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu'))
    
    #C5, P5
    model.add(Conv2D(64, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    
    #F6, F7, F8
    model.add(Dense(128, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_class, activation='softmax'))
    
    
    
    
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    
    #valid_in, valid_out = generate_batch_data(valid_image_paths, valid_labels)
    #test_in, test_out = generate_batch_data(test_image_paths, test_labels)
    #print("Finished generating data for validation and test.")
    
    model.fit_generator(generate_batch_data(train_image_paths, train_labels, batch_size),
                    steps_per_epoch = int(len(train_image_paths) / batch_size), epochs=10, 
                    validation_data = generate_batch_data(valid_image_paths, valid_labels, batch_size), 
                    validation_steps = int(len(valid_labels)/batch_size))
    
#     trained_epoch = 0
#     for e in range(0, epoch):
#         print("Epoch:", e)
#         for i in range (1, len(train_image_paths), batch_size):
           
            
#             input, output = generate_batch_data(train_image_paths[i:i+batch_size], train_labels[i:i+batch_size])

#             model.train_on_batch(input, output)
#             loss_train = model.test_on_batch(input, output)
#           #  loss_valid = model.test_on_batch(valid_in, valid_out)
#             print("loss in batch of training set:", loss_train)
#            # print("loss in validation sset:", loss_valid)

        
    #Test set
#     predictions = model.predict_on_batch(test_in)
    
#     true_predict = 0
#     for predict, label in zip(predictions, test_out):
#         if (predict == label):
#             true_predict = true_predict +1
    
#     print("Accuracy in test set: ", true_predict / len(predictions))
            
    
        
        
    
  
    #Your statements here

    stop = timeit.default_timer()

    print ("Time for training and testing: ", stop - start, "(s)") 